In [1]:
import itertools
import gc
from bib import *
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import backend as K 
from keras import  activations
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Dense, Activation, regularizers,BatchNormalization,Dropout
import tensorflow as tf
from  tqdm import tqdm_notebook
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
eps=np.finfo(np.float32).eps
xtrains,ytrains,xvals,yvals,xtest,ytest=split(('CHL 1','THERM 1','SSH','SR','WS','5days','longitude','latitude'),
                                               ['CHL '+ str(i) for i in range(2,19)])
ytrains,yvals,ytest=applyToy(lambda x:np.log(x)/np.log(10),ytrains,yvals,ytest)

In [3]:
for xt,xv in zip(xtrains,xvals):
    xt['cosT']=np.cos(np.pi*2*xt['5days']/73)
    xt['sinT']=np.sin(np.pi*2*xt['5days']/73)
    xv['cosT']=np.cos(np.pi*2*xv['5days']/73)
    xv['sinT']=np.sin(np.pi*2*xv['5days']/73)
    
    
    xt['t.5d']=xt['THERM 1']*xt['5days']
    xv['t.5d']=xv['THERM 1']*xv['5days']
    xt['t.5d']=xt['THERM 1']*xt['latitude']
    xv['t.5d']=xv['THERM 1']*xv['latitude']
    
    
    xt.drop('5days', 1,inplace=True)
    xv.drop('5days', 1,inplace=True)

    
    
    
xtest['cosT']=np.cos(np.pi*2*xtest['5days']/73)
xtest['sinT']=np.sin(np.pi*2*xtest['5days']/73)
xtest.drop('5days', 1,inplace=True)

In [7]:

def generate(arch,act):
    layers=[]
    for index,size in enumerate(arch):
        if index ==0:
            layers.append(Dense(size, input_shape=(10,),activation=act[index]))
#             layers.append(BatchNormalization())

        else:
            layers.append(Dense(size,activation=act[index]))
#             layers.append(Dropout(0.3))
    layers.append(Dense(17,activation='linear',kernel_regularizer=regularizers.l2(0.01)))
    model = Sequential(layers)
    model.compile(optimizer='Adam',
              loss='mse')
    return model
callbacks = [EarlyStopping(monitor='val_loss', patience=15)]

In [8]:
def validate(arch,act,xtrains,ytrains,xvals,yvals):
    fold_evaluation=[]
    for xt,yt,xv,yv in tqdm_notebook(zip(xtrains,ytrains,xvals,yvals),total=len(xtrains)):
        model=generate(arch,act)
        
        mean=xt.mean()
        std=xt.std()
        
        xt=(xt-mean)/std
        xv=(xv-mean)/std
        
        
#         yt=np.exp(yt*np.log(10))-eps
#         yv=np.exp(yv*np.log(10))-eps
        
        
        model.fit(xt.values,yt.values,
                  epochs=10000,
                  callbacks=callbacks,
                  verbose=0,
                  batch_size=73*10, 
                  validation_data=(xv.values,yv.values))
        
        yp = model.predict(xv.values)
        
        #check error in original space
        
        yp=np.exp(yp*np.log(10))
        yv=np.exp(yv*np.log(10))       
        fold_evaluation.append(evaluate(yp,yv))
        print(evaluate(yp,yv))
        
    return np.mean(fold_evaluation),np.std(fold_evaluation )


In [11]:
arch=(1000, 500, 500, 500, 1000, 2000)
K.clear_session()
print('ARCH :'+ str(arch)) 
# print('ACT :'+ str(act))
error,std=validate(arch,['softplus']*len(arch),xtrains,ytrains,xvals,yvals)
print('\t error: %.5f +- %.5f' % (error,std))

ARCH :(1000, 500, 500, 500, 1000, 2000)
0.05963295032761503
0.061527533069926095
0.058198062935206414
0.10962035905821053
0.09729015839003431
0.06527183408117691
0.0423510352414553
0.06591719539368046
0.047828707651910364
0.03800254466227275
0.04161946417921301
0.07145351261827264
0.03387871669742635
0.05672203472582175
0.05349774247740088
0.0503072701315018

	 error: 0.05957 +- 0.01962


cos+sin 5days
ARCH :(1000, 500, 500, 500, 1000, 2000)
    error: 0.05983 +- 0.02043
ARCH :(1000, 500, 500, 500, 1000, 2000)
    error: 0.05957 +- 0.01907
cos+sin 5days + year
ARCH :(1000, 500, 500, 500, 1000, 2000)
     error: 0.05895 +- 0.01805
	 error: 0.05697 +- 0.01715
cos+sin 5days + year+ space
ARCH :(1000, 500, 500, 500, 1000, 2000)
      error: 0.05603 +- 0.01869
      
cos+sin 5days +space
ARCH :(1000, 500, 500, 500, 1000, 2000)
     error: 0.05930 +- 0.01841
cos+sin 5days + atitude no ws     
     error: 0.05950 +- 0.01988

+ vars
error: 0.05868 +- 0.02006
error: 0.05727 +- 0.01928
error: 0.05957 +- 0.01962